In [1]:
import pandas as pd
import json
import pybedtools
import pathlib
import subprocess

In [2]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
with open(
        '/home/hanliu/project/mouse_rostral_brain/metadata/label_transfer_to_atac_new_version/mC-ATAC.CellClass.json'
) as f:
    cell_class_to_atac = json.load(f)

with open(
        '/home/hanliu/project/mouse_rostral_brain/metadata/label_transfer_to_atac_new_version/mC-ATAC.MajorType.json'
) as f:
    major_type_to_atac = json.load(f)

with open(
        '/home/hanliu/project/mouse_rostral_brain/metadata/label_transfer_to_atac_new_version/mC-ATAC.SubType.json'
) as f:
    sub_type_to_atac = json.load(f)

In [12]:
atac_peak_dir = pathlib.Path('/home/hanliu/project/mouse_rostral_brain/ATAC/New/SubType/bed/')
atac_paths = {'.'.join(path.name.split('.')[:3]): path for path in atac_peak_dir.glob('*bed')}

In [15]:
outdir = 'v2/CellClass/'
pathlib.Path(outdir).mkdir(exist_ok=True)

for cluster, atac_cluster_list in cell_class_to_atac.items():
    atac_files = [str(atac_paths[c]) for c in atac_cluster_list]
    atac_file_str = ' '.join(atac_files)
    subprocess.run(f'cat {atac_file_str} > {outdir}/{cluster}.peak.bed',
                   shell=True)
    subprocess.run(
        f'sort --parallel=20 -S 50G -k 1,1 -k2,2n {outdir}/{cluster}.peak.bed > {outdir}/{cluster}.sorted.bed',
        shell=True)
    subprocess.run(
        f'bedtools merge -i {outdir}/{cluster}.sorted.bed > {outdir}/{cluster}.sorted.peak.bed',
        shell=True)
    subprocess.run(
        f'rm -f {outdir}/{cluster}.sorted.bed {outdir}/{cluster}.peak.bed',
        shell=True)

In [16]:
outdir = 'v2/MajorType/'
pathlib.Path(outdir).mkdir(exist_ok=True)

for cluster, atac_cluster_list in major_type_to_atac.items():
    atac_files = [str(atac_paths[c]) for c in atac_cluster_list]
    if len(atac_files) == 0:
        continue
    atac_file_str = ' '.join(atac_files)
    subprocess.run(f'cat {atac_file_str} > {outdir}/{cluster}.peak.bed',
                   shell=True)
    subprocess.run(
        f'sort --parallel=20 -S 50G -k 1,1 -k2,2n {outdir}/{cluster}.peak.bed > {outdir}/{cluster}.sorted.bed',
        shell=True)
    subprocess.run(
        f'bedtools merge -i {outdir}/{cluster}.sorted.bed > {outdir}/{cluster}.sorted.peak.bed',
        shell=True)
    subprocess.run(
        f'rm -f {outdir}/{cluster}.sorted.bed {outdir}/{cluster}.peak.bed',
        shell=True)

In [17]:
outdir = 'v2/SubType/'
pathlib.Path(outdir).mkdir(exist_ok=True)

for cluster, atac_cluster_list in sub_type_to_atac.items():
    cluster = cluster.replace(' ', '_')
    atac_files = [str(atac_paths[c]) for c in atac_cluster_list]
    if len(atac_files) == 0:
        continue
    atac_file_str = ' '.join(atac_files)
    subprocess.run(f'cat {atac_file_str} > {outdir}/{cluster}.peak.bed',
                   shell=True, check=True)
    subprocess.run(
        f'sort --parallel=20 -S 50G -k 1,1 -k2,2n {outdir}/{cluster}.peak.bed > {outdir}/{cluster}.sorted.bed',
        shell=True, check=True)
    subprocess.run(
        f'bedtools merge -i {outdir}/{cluster}.sorted.bed > {outdir}/{cluster}.sorted.peak.bed',
        shell=True, check=True)
    subprocess.run(
        f'rm -f {outdir}/{cluster}.sorted.bed {outdir}/{cluster}.peak.bed',
        shell=True, check=True)